In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [2]:
from fnsa.lexicon import Lexicon
from fnsa.util import *
import spacy

In [3]:
nlp = spacy.load('en_core_web_sm')
sentences = load_sentences()

In [14]:
lexicon = Lexicon(nlp)

In [15]:
for i, sentence in enumerate(sentences[:1]):
    doc = lexicon(sentence)
    show(doc, index=(i+1), include_text=True)
    print("-----")

[('fe', '+', 4, 5, -1, False), ('if', 'rev', 8, 9, -1, False), ('if', 'mod', 5, 6, -1, False), ('if', 'mod', 6, 7, -1, False), ('lm', '+', 5, 6, -1, False), ('lm', '-', 8, 9, -1, False), ('xe', 'org', 1, 2, -1, True), ('xe', 'org', 4, 5, -1, True)]
1. The Group's operative EBIT will probably be negative.
   The [Group/xe:org] 's operative [EBIT/fe:+] [will/if:mod] [probably/if:mod] be [negative/if:rev] .
-----


In [16]:
for lex, category, start, end, index, acceptable in doc.user_data['entries']:
    print("%s --> %s/%s" % (doc[start:end], lex, category))

Group --> xe/org
probably --> if/mod
EBIT --> fe/+
negative --> if/rev
will --> if/mod
